# How to translate iTable tables to API queries
In some areas the API and the iTables are quite similar (e.g., in National and Regional), but in others they are organized somewhat differently. This guide shows how to translate the process of getting data from various iTables searches to using the API. We do this for each of the datasets.

## General info

Note that for some scenarios it may take several API queries to generate the data for a single iTables table. In some cases, even when the data returned is broadly similar, there may be subtle differences (e.g., a National table may have additional "Addenda" rows in iTables that are not present in the API <!--NIPA:T10403-->; the series titles in iTables may have prefixes, such as "Less: "/"Equals: ", to show how rows aggregate to the total that are are not present in the API/Series Register<!--NIPA:T10405 -->; in some cases lines with missing data will be omitted in the API that appear in iTables).

Also, in the examples below we often search for a target parameter by a string value in a table. You can always export the table of possibilities to spreadsheet to view the whole thing (e.g., `tbl.to_csv("fname.csv", index=False)`) 

We first need to figure out what dataset to pull from. Here's roughly how the "tabs" on the iTables web page map to the different API datasets:

- "(National Data) GDP & Personal Income" -> [NIPA, NIUnderlyingDetail]
- "Fixed Assets" -> FixedAssets
- "GDP by Industry" -> [GDPbyIndustry, UnderlyingGDPbyIndustry]
- "Input-Output" -> InputOutput
- "Int'l Transactions, Services, & IIP" -> [ITA, [IntlServTrade, IntlServSTA], IIP]
- "Direct Investments & MNEs" -> MNE
- "(Regional Data) GDP & Personal Income" -> Regional

## Other resources
While we show one method here, there are some other resources that may be helpful:

- `search_metadata()` can be be used for 'FixedAssets', 'NIPA', 'NIUnderlyingDetail'. 
- NIPA web resources: [Table Title to TableID](https://apps.bea.gov/national/Release/TXT/TablesRegister.txt), [Series Details to Series Code](https://apps.bea.gov/national/Release/TXT/SeriesRegister.txt), [NIPA keywords](https://apps.bea.gov/iTable/iTable.cfm?ReqID=13&step=1)
- Fixed Assets web resources: [Table Register](https://apps.bea.gov/national/FixedAssets/Release/TXT/TablesRegister.txt), [Series Register](https://apps.bea.gov/national/FixedAssets/Release/TXT/SeriesRegister.txt),
- [Regional API PDF](https://apps.bea.gov/regional/pdf/RegionalApi.pdf) has table mapping 



## Setup

In [1]:
import beaapi

import pandas as pd
pd.set_option('display.max_colwidth', None)  # show all text in cells

In [2]:
# Get key from unversioned file
from dotenv import dotenv_values
beakey = dotenv_values()["beakey"]

### Code utilities
We define here a few tools to be use below

In [5]:
# Get some basic Meta-data. What are the table names?
datasets_info = beaapi.get_data_set_list(beakey)
dataset_names = list(datasets_info['DatasetName'])
param_infos = {dataset_name: beaapi.get_parameter_list(beakey, dataset_name) for dataset_name in dataset_names}

Make a function `get_table_param_from_desc()` to ease getting table IDs from descriptions

In [6]:
# Tools to help lookup table IDs from table descriptions
table_var = {'NIPA':'TableName',
    'NIUnderlyingDetail':'TableName',
    'MNE':None,
    'FixedAssets': 'TableName',
    'ITA':None,
    'IIP':None,
    'InputOutput':"TableID",
    'IntlServTrade':None,
    'IntlServSTA':None,
    'GDPbyIndustry':"TableID",
    'Regional': "TableName",
    'UnderlyingGDPbyIndustry':"TableID"}
table_param_desc = {'NIPA':'Description',
    'NIUnderlyingDetail':'Description',
    'MNE':None,
    'FixedAssets': 'Description',
    'ITA':None,
    'IIP':None,
    'InputOutput':"Desc",
    'IntlServTrade':None,
    'GDPbyIndustry':"Desc",
    'Regional': "Desc",
    'UnderlyingGDPbyIndustry':"Desc"}
table_names = {}
for dataset_name, table_var_name in table_var.items():
    if table_var_name is not None:
        table_names[dataset_name] = beaapi.get_parameter_values(beakey, dataset_name, table_var_name)

def get_table_param_from_desc(dataset_name, desc):
    mask = table_names[dataset_name][table_param_desc[dataset_name]].str.contains(desc)
    results = table_names[dataset_name][mask]
    table_var_name = table_var[dataset_name]
    return table_var_name, results

## How to build query from iTables

### National Data - GDP & Personal Income

#### NIPA (publication category "GDP and Personal Income", default)

First, let's take as example table the first one on the web page and describe what we do have

In [39]:
# Example
desc = "Table 1.1.1. Percent Change From Preceding Period in Real Gross Domestic Product"
freq = "Q"
year = "2019" # input as string

Then let's see what we need to build the query

In [40]:
display(param_infos["NIPA"])

ParameterName ParameterDataType  \
0     Frequency            string   
1  ShowMillions            string   
2       TableID           integer   
3     TableName            string   
4          Year           integer   

                                ParameterDescription  ParameterIsRequiredFlag  \
0                 A - Annual, Q-Quarterly, M-Monthly                        1   
1  A flag indicating that million-dollar data sho...                        0   
2                 The standard NIPA table identifier                        0   
3                      The new NIPA table identifier                        0   
4    List of year(s) of data to retrieve (X for All)                        1   

  ParameterDefaultValue  MultipleAcceptedFlag AllValue  
0                                           1           
1                     N                     0           
2                  <NA>                     0           
3                  <NA>                     0           
4                                           1        X

Note, we actually need exactly 1 of TableID or TableName. (They transitioned to TableName, but keep TableID around for compatibility.)

Next let's find the TableName from the description.

In [41]:
table_param_name, table_param_vals = get_table_param_from_desc("NIPA", desc)
display(table_param_vals)
print("We'll use that one")
table_param_val = table_param_vals.iloc[0,0]


TableName                                        Description
0    T10101  Table 1.1.1. Percent Change From Preceding Per...

We'll use that one


Finally, let's get the data.

In [42]:
tbl = beaapi.get_data(beakey, "NIPA", TableName=table_param_val, Frequency=freq, Year=year)
tbl.head()

TableName SeriesCode  LineNumber                    LineDescription  \
0    T10101     A191RL           1             Gross domestic product   
1    T10101     A191RL           1             Gross domestic product   
2    T10101     A191RL           1             Gross domestic product   
3    T10101     A191RL           1             Gross domestic product   
4    T10101     DPCERL           2  Personal consumption expenditures   

  TimePeriod            METRIC_NAME                      CL_UNIT  UNIT_MULT  \
0     2019Q1  Fisher Quantity Index  Percent change, annual rate          0   
1     2019Q2  Fisher Quantity Index  Percent change, annual rate          0   
2     2019Q3  Fisher Quantity Index  Percent change, annual rate          0   
3     2019Q4  Fisher Quantity Index  Percent change, annual rate          0   
4     2019Q1  Fisher Quantity Index  Percent change, annual rate          0   

   DataValue NoteRef  
0        2.4  T10101  
1        3.2  T10101  
2        2.8  T10101  
3        1.9  T10101  
4        0.6  T10101

#### NIUnderlyingDetail (Publication Category "Underlying Details", non-default)

In [12]:
# Info from iTables
desc = "Table 1AU. Real Manufacturing and Trade Inventories, Seasonally Adjusted, End of Period" # Note: have to cut off final "[Chained 1996 dollars, 1967-96, SIC]" to get it to match
freq="Q"
year="1995"

print("What parameters are needed")
display(param_infos["NIUnderlyingDetail"])

What parameters are needed


ParameterName ParameterDataType  \
0     Frequency            string   
1       TableID           integer   
2     TableName            string   
3          Year           integer   

                                ParameterDescription  ParameterIsRequiredFlag  \
0                 A - Annual, Q-Quarterly, M-Monthly                        1   
1  The standard NI underlying detail table identi...                        0   
2                      The new NIPA table identifier                        0   
3    List of year(s) of data to retrieve (X for All)                        1   

  ParameterDefaultValue  MultipleAcceptedFlag AllValue  
0                                           1           
1                  <NA>                     0           
2                  <NA>                     0           
3                                           1        X

Get tablename from iTable's table description from web

In [13]:
table_param_name, table_param_vals = get_table_param_from_desc("NIUnderlyingDetail", desc)
display(table_param_vals)
print("That one looks good.")
table_param_val = table_param_vals.iloc[0,0]

What TableName matches?


TableName                                        Description
0     U001A  Table 1AU. Real Manufacturing and Trade Invent...

Next, get data.

In [14]:
tbl = beaapi.get_data(beakey, "NIUnderlyingDetail", TableName=table_param_val, Frequency=freq, Year=year)
tbl.head()

TableName SeriesCode  LineNumber          LineDescription TimePeriod  \
0     U001A      KSTMT           1  Manufacturing and trade     1995Q1   
1     U001A      KSTMT           1  Manufacturing and trade     1995Q2   
2     U001A      KSTMT           1  Manufacturing and trade     1995Q3   
3     U001A      KSTMT           1  Manufacturing and trade     1995Q4   
4     U001A       KSTM           2            Manufacturing     1995Q1   

       METRIC_NAME CL_UNIT  UNIT_MULT  DataValue NoteRef  
0  Chained Dollars   Level          6     973434   U001A  
1  Chained Dollars   Level          6     984218   U001A  
2  Chained Dollars   Level          6     991162   U001A  
3  Chained Dollars   Level          6     994259   U001A  
4  Chained Dollars   Level          6     410079   U001A

In [10]:
# Unsure how to access Publication category "Additional Tables"
# desc="FWPI-1. Fixed-Weighted Price Indexes"
# Both methods fail:
#search_result = beaapi.search_metadata(desc, beakey)
#display(search_result)
#table_param_name, table_param_vals = get_table_param_from_desc("NIPA", desc) #Tried with NIUnderlyingDetail
#display(table_param_vals)


### National Data - Fixed Assets

#### FixedAssets

In [22]:
# Info from iTable
desc = "Table 1.1. Current-Cost Net Stock of Fixed Assets and Consumer Durable Goods"
year="2013"

First, what parameters are needed?

In [18]:
display(param_infos["FixedAssets"])

ParameterName ParameterDataType  \
0     TableName            string   
1          Year           integer   

                              ParameterDescription  ParameterIsRequiredFlag  \
0                  The new Fixed Assets identifier                        1   
1  List of year(s) of data to retrieve (X for All)                        1   

  ParameterDefaultValue  MultipleAcceptedFlag AllValue  
0                                           0           
1                                           1        X

Get tablename from iTable's table description from web

In [24]:
table_param_name, table_param_vals = get_table_param_from_desc("FixedAssets", desc)
display(table_param_vals)
print("That one looks good")
table_param_val = table_param_vals.iloc[0,0]

TableName                                        Description
0   FAAt101  Table 1.1. Current-Cost Net Stock of Fixed Ass...

Get data

In [25]:
tbl = beaapi.get_data(beakey, "FixedAssets", TableName=table_param_val, Year=year)
tbl.head()

TableName    SeriesCode  LineNumber  \
0   FAAt101  k1wtotl1es00           1   
1   FAAt101  k1ttotl1es00           2   
2   FAAt101  k1ptotl1es00           3   
3   FAAt101  k1ntotl1es00           4   
4   FAAt101  k1ntotl1eq00           5   

                           LineDescription TimePeriod      METRIC_NAME  \
0  Fixed assets and consumer durable goods       2013  Current Dollars   
1                             Fixed assets       2013  Current Dollars   
2                                  Private       2013  Current Dollars   
3                           Nonresidential       2013  Current Dollars   
4                                Equipment       2013  Current Dollars   

  CL_UNIT  UNIT_MULT  DataValue  NoteRef  
0   Level          9    56290.3  FAAt101  
1   Level          9    51510.7  FAAt101  
2   Level          9    38567.6  FAAt101  
3   Level          9    21073.3  FAAt101  
4   Level          9     5897.2  FAAt101

### Industry Data

#### GDPbyIndustry (Publication category "GDP-by-Industry", a default)

In [28]:
# Info from iTable
desc = "Value Added by Industry"
year="2019"
freq="Q"
industries = "ALL" # from iTable

First, what parameters are needed?

In [29]:
display(param_infos["GDPbyIndustry"])

ParameterName ParameterDataType  \
0     Frequency            string   
1      Industry            string   
2       TableID           integer   
3          Year           integer   

                                ParameterDescription  ParameterIsRequiredFlag  \
0                            A - Annual, Q-Quarterly                        1   
1       List of industries to retrieve (ALL for All)                        1   
2  The unique GDP by Industry table identifier (A...                        1   
3  List of year(s) of data to retrieve (ALL for All)                        1   

  ParameterDefaultValue  MultipleAcceptedFlag AllValue  
0                                           1      ALL  
1                                           1      ALL  
2                                           1      ALL  
3                                           1      ALL

Get tablename

In [30]:
table_param_name, table_param_vals = get_table_param_from_desc("GDPbyIndustry", desc)
display(table_param_vals)
print("That first one looks right.")
table_param_val = table_param_vals.iloc[0,0]

What TableName matches?


Key                                               Desc
0   1                    Value Added by Industry (A) (Q)
2   6          Components of Value Added by Industry (A)
3   7  Components of Value Added by Industry as a Per...
4   8  Chain-Type Quantity Indexes for Value Added by...
5   9  Percent Changes in Chain-Type Quantity Indexes...
6  10               Real Value Added by Industry (A) (Q)
7  11  Chain-Type Price Indexes for Value Added by In...
8  12  Percent Changes in Chain-Type Price Indexes fo...

Get data

In [31]:
tbl = beaapi.get_data(beakey, "GDPbyIndustry", TableID=table_param_val, Year=year, Frequency=freq, Industry=industries)
tbl.head()

TableID Frequency  Year Quarter Industry  \
0        1         Q  2019       I       11   
1        1         Q  2019       I    111CA   
2        1         Q  2019       I    113FF   
3        1         Q  2019       I       21   
4        1         Q  2019       I      211   

                           IndustrYDescription  DataValue NoteRef  
0  Agriculture, forestry, fishing, and hunting      159.7       1  
1                                        Farms      120.9       1  
2    Forestry, fishing, and related activities       38.8       1  
3                                       Mining      302.1       1  
4                       Oil and gas extraction      185.6       1

#### UnderlyingGDPbyIndustry  (Publication category "Underlying", non-default)

In [35]:
# Info from iTable
desc = "U.Value Added by Industry"
year="2013"
freq="A"

What parameters are needed?

In [36]:
display(param_infos["UnderlyingGDPbyIndustry"])
industries = "ALL" # from iTable

ParameterName ParameterDataType  \
0     Frequency            string   
1      Industry            string   
2       TableID           integer   
3          Year           integer   

                                ParameterDescription  ParameterIsRequiredFlag  \
0                                        Q-Quarterly                        1   
1       List of industries to retrieve (ALL for All)                        1   
2  The unique Underlying GDP by Industry table id...                        1   
3  List of year(s) of data to retrieve (ALL for All)                        1   

  ParameterDefaultValue  MultipleAcceptedFlag AllValue  
0                                           1      ALL  
1                                           1      ALL  
2                                           1      ALL  
3                                           1      ALL

Get tablename

In [37]:
table_param_name, table_param_vals = get_table_param_from_desc("UnderlyingGDPbyIndustry", desc)
print("What TableName matches?")
display(table_param_vals)
table_param_val = table_param_vals.iloc[0,0]
print("First one")

What TableName matches?


Key                           Desc
0  210  U.Value Added by Industry (A)

First one


Get data

In [38]:
tbl = beaapi.get_data(beakey, "UnderlyingGDPbyIndustry", TableID=table_param_val, Year=year, Frequency=freq, Industry=industries)
tbl.head()

TableID Frequency  Year Industry  \
0      210         A  2013       11   
1      210         A  2013      111   
2      210         A  2013    111CA   
3      210         A  2013      112   
4      210         A  2013    113FF   

                           IndustrYDescription  DataValue NoteRef  
0  Agriculture, forestry, fishing, and hunting      214.3     210  
1                              Crop production      111.7     210  
2                                        Farms      183.3     210  
3            Animal production and aquaculture       71.5     210  
4    Forestry, fishing, and related activities       31.0     210

#### InputOutput  (Publication category "Input-Output", a default)

In [43]:
# Info from iTable
desc = "The Use of Commodities by Industries - Sector"
year="2020"

print("What parameters are needed")
display(param_infos["InputOutput"])
#industries = "ALL" # from iTable

What parameters are needed


ParameterName ParameterDataType  \
0       TableID           integer   
1          Year           integer   

                                ParameterDescription  ParameterIsRequiredFlag  \
0           The unique Input-Output table identifier                        1   
1  List of year(s) of data to retrieve (ALL for All)                        1   

  ParameterDefaultValue  MultipleAcceptedFlag AllValue  
0                                           1           
1                                           1      ALL

Get tablename

In [44]:
table_param_name, table_param_vals = get_table_param_from_desc("InputOutput", desc)
print("What TableName matches?")
display(table_param_vals)
print("That first one looks good")
table_param_val = table_param_vals.iloc[0,0]

What TableName matches?


Key                                           Desc
9  258  The Use of Commodities by Industries - Sector

That first one looks good


Get data

In [45]:
tbl = beaapi.get_data(beakey, "InputOutput", TableID=table_param_val, Year=year)
tbl.head()

TableID  Year RowCode                                     RowDescr  \
0      258  2020      11  Agriculture, forestry, fishing, and hunting   
1      258  2020      11  Agriculture, forestry, fishing, and hunting   
2      258  2020      11  Agriculture, forestry, fishing, and hunting   
3      258  2020      11  Agriculture, forestry, fishing, and hunting   
4      258  2020      11  Agriculture, forestry, fishing, and hunting   

     RowType ColCode                                           ColDescr  \
0  Commodity      11        Agriculture, forestry, fishing, and hunting   
1  Commodity    T001                                 Total Intermediate   
2  Commodity     31G                                      Manufacturing   
3  Commodity       7  Arts, entertainment, recreation, accommodation...   
4  Commodity      21                                             Mining   

    ColType  DataValue NoteRef  
0  Industry     106419     258  
1  Industry     446013     258  
2  Industry     309395     258  
3  Industry       8556     258  
4  Industry        168     258

In [15]:

# Unsure how to access Publication category "Make Use Framework"
# desc="The Use of Commodities by Industries, After Redefinitions"
# Both methods fail:
#search_result = beaapi.search_metadata(desc, beakey)
#display(search_result)
#table_param_name, table_param_vals = get_table_param_from_desc("UnderlyingGDPbyIndustry", desc) # Tried also with InputOutput, GDPbyIndustry
#display(table_param_vals)

### Int'l Transactions, Services, IIP

#### ITA

The different tables are associated with different sets of lines (indicators). So some tables aren't easy to reconstruct exactly.

Note: (in PDF but API error msg confusing) you can't get all indicators using "AllCountries" (agg'd over countries), even though that would seem easy (and is something like like table 1.1).

Basically you have to look through `beaapi.get_parameter_values(beakey, "ITA", "Indicator")`

Note also that the following indicators don't report global sum, so ask for all countries individually: 'ExpGdsAutoEngAndEngParts', 'ExpGdsOthAutoPartsAndAcc', 'ExpGdsPassCars', 'ExpGdsTrucksBusesSpecPurpVeh',  'ImpGdsAutoEngAndEngParts', 'ImpGdsOthAutoPartsAndAcc',  'ImpGdsPassCars', 'ImpGdsTrucksBusesSpecPurpVeh'

In [49]:
# Roughly one line from "Table 1.2. U.S. International Transactions, Expanded Detail"
year="2020"
freq="QSA"
cntry="AllCountries"
desc = "Balance on goods"

print("What parameters are needed")
display(param_infos["ITA"])

What parameters are needed


ParameterName ParameterDataType  \
0      Indicator            string   
1  AreaOrCountry            string   
2      Frequency            string   
3           Year            string   

                                ParameterDescription  ParameterIsRequiredFlag  \
0  The indicator code for the type of transaction...                        0   
1                      The area or country requested                        0   
2  A - Annual, QSA - Quarterly seasonally adjuste...                        0   
3                                     Year requested                        0   

  ParameterDefaultValue  MultipleAcceptedFlag AllValue  
0                   ALL                     1      ALL  
1          AllCountries                     1      ALL  
2                   ALL                     1      ALL  
3                   ALL                     1      ALL

Get indicator (or get a single country)

In [51]:
# areas_ITA = beaapi.get_parameter_values(beakey, 'ITA', 'AreaOrCountry')
ita_indicatorsID = beaapi.get_parameter_values(beakey, 'ITA', 'Indicator')

ind_result = ita_indicatorsID[ita_indicatorsID["Desc"].str.contains(desc)]
display(ind_result)
print("Take the first one")
indicator = "BalGds"

Key                           Desc
2      BalGds               Balance on goods
3  BalGdsServ  Balance on goods and services

Take the first one


Get data

In [52]:
tbl = beaapi.get_data(beakey, "ITA", Year=year, AreaOrCountry=cntry, Frequency=freq, Indicator=indicator)
tbl.head()

Indicator AreaOrCountry Frequency  Year       TimeSeriesId  \
0    BalGds  AllCountries       QSA  2020  TSI_ItaBalGds_QSA   
1    BalGds  AllCountries       QSA  2020  TSI_ItaBalGds_QSA   

                             TimeSeriesDescription TimePeriod CL_UNIT  \
0  Balance on goods; quarterly seasonally adjusted     2020Q3     USD   
1  Balance on goods; quarterly seasonally adjusted     2020Q4     USD   

  UNIT_MULT  DataValue NoteRef  
0         6    -245370       Q  
1         6    -253125       Q

#### IntlServTrade

Similar issue about different iTables tables being different sets or broken out by country in different ways. Basically have to look at TypeOfService values to line up with tables. Though in this case we can get all types of service of "AllCountries" (agg'd over countries)

In [56]:
type_desc = "Sea transport"

print("What parameters are needed")
display(param_infos["IntlServTrade"])


What parameters are needed


ParameterName ParameterDataType           ParameterDescription  \
0   TypeOfService            string  The type of service requested   
1  TradeDirection            string  The trade direction requested   
2     Affiliation            string      The affiliation requested   
3   AreaOrCountry            string  The area or country requested   
4            Year            string             The year requested   

   ParameterIsRequiredFlag ParameterDefaultValue  MultipleAcceptedFlag  \
0                        0                   ALL                     1   
1                        0                   ALL                     1   
2                        0                   ALL                     1   
3                        0          AllCountries                     1   
4                        0                   ALL                     1   

  AllValue  
0      ALL  
1      ALL  
2      ALL  
3      ALL  
4      ALL

Get type

In [58]:
# cntr_IST = beaapi.get_parameter_values(beakey, 'IntlServTrade', 'AreaOrCountry')
ist_Type = beaapi.get_parameter_values(beakey, 'IntlServTrade', 'TypeOfService')
type_result = ist_Type[ist_Type["Desc"].str.contains(type_desc)]
display(type_result)
type_of_service = type_result.iloc[0,0]

Key                    Desc
104  TransportSea  Sea transport services

Get data

In [59]:
tbl = beaapi.get_data(beakey, "IntlServTrade", AreaOrCountry="AllCountries",TypeOfService=type_of_service, Year="1999")
tbl.head()

TypeOfService TradeDirection      Affiliation AreaOrCountry  Year  \
0  TransportSea        Exports       Affiliated  AllCountries  1999   
1  TransportSea        Exports  AllAffiliations  AllCountries  1999   
2  TransportSea        Exports     Unaffiliated  AllCountries  1999   
3  TransportSea        Exports     UsAffiliates  AllCountries  1999   
4  TransportSea        Exports        UsParents  AllCountries  1999   

                                 TimeSeriesId  \
0     TSI_IstTransportSeaExpAllCountriesAff_A   
1  TSI_IstTransportSeaExpAllCountriesAllAff_A   
2   TSI_IstTransportSeaExpAllCountriesUnaff_A   
3   TSI_IstTransportSeaExpAllCountriesUsAff_A   
4   TSI_IstTransportSeaExpAllCountriesUsPar_A   

                               TimeSeriesDescription TimePeriod CL_UNIT  \
0  Exports of sea transport services; all countri...       1999     USD   
1   Exports of sea transport services; all countries       1999     USD   
2  Exports of sea transport services; all countri...       1999     USD   
3  Exports of sea transport services; all countri...       1999     USD   
4  Exports of sea transport services; all countri...       1999     USD   

  UNIT_MULT  DataValue NoteRef  
0         6        NaN   .....  
1         6    10489.0          
2         6    10489.0          
3         6        NaN   .....  
4         6        NaN   .....

#### IntlServSTA
Use `channel="Mofas"` or `="Mousas"` to get the data in tables 4.1-4 and 5.1-4.

In [7]:
print("What parameters are needed")
display(param_infos["IntlServSTA"])

tbl = beaapi.get_data(beakey, "IntlServSTA", Channel="Mofas")
tbl.head()

What parameters are needed


ParameterName ParameterDataType           ParameterDescription  \
0        Channel            string          The channel requested   
1    Destination            string      The destination requested   
2       Industry            string         The industry requested   
3  AreaOrCountry            string  The area or country requested   
4           Year            string             The year requested   

   ParameterIsRequiredFlag ParameterDefaultValue  MultipleAcceptedFlag  \
0                        0                   ALL                     1   
1                        0                   ALL                     1   
2                        0                   ALL                     1   
3                        0          AllCountries                     1   
4                        0                   ALL                     1   

  AllValue  
0      ALL  
1      ALL  
2      ALL  
3      ALL  
4      ALL

Channel Destination                                 Industry AreaOrCountry  \
0   Mofas  AllForeign                            Accommodation  AllCountries   
1   Mofas  AllForeign                     AccommodationAndFood  AllCountries   
2   Mofas  AllForeign   AccountingTaxPrepBookkeepingAndPayroll  AllCountries   
3   Mofas  AllForeign  AdministrationSupportAndWasteManagement  AllCountries   
4   Mofas  AllForeign                 AdministrativeAndSupport  AllCountries   

   Year  \
0  2020   
1  2020   
2  2020   
3  2020   
4  2020   

                                                                   TimeSeriesId  \
0                            TSI_SstaMofasAllForeignAccommodationAllCountries_A   
1                     TSI_SstaMofasAllForeignAccommodationAndFoodAllCountries_A   
2   TSI_SstaMofasAllForeignAccountingTaxPrepBookkeepingAndPayrollAllCountries_A   
3  TSI_SstaMofasAllForeignAdministrationSupportAndWasteManagementAllCountries_A   
4                 TSI_SstaMofasAllForeignAdministrativeAndSupportAllCountries_A   

                                                                                                                       TimeSeriesDescription  \
0                                                   Services supplied through MOFAs in accommodation in all countries to all foreign persons   
1                                 Services supplied through MOFAs in accommodation and food services in all countries to all foreign persons   
2  Services supplied through MOFAs in accounting, tax preparation, bookkeeping, and payroll services in all countries to all foreign persons   
3                   Services supplied through MOFAs in administration, support, and waste management in all countries to all foreign persons   
4                             Services supplied through MOFAs in administrative and support services in all countries to all foreign persons   

  TimePeriod CL_UNIT UNIT_MULT  DataValue NoteRef  
0       2020     USD         6    10154.0          
1       2020     USD         6    36820.0          
2       2020     USD         6    13904.0          
3       2020     USD         6    68465.0          
4       2020     USD         6    65017.0

#### Services Trade by Enterprise Characteristics (STEC)
Not currently available through the API

#### IIP

Similar issues as above, but now `TypeOfInvestment` are the lines in tables. Does allow all `TypeOfInvestment` for "AllCountries" (agg'd over countries)

In [61]:
type_desc = "Net international investment position excluding financial derivatives"

print("What parameters are needed")
display(param_infos["IIP"])

What parameters are needed


ParameterName ParameterDataType  \
0  TypeOfInvestment            string   
1         Component            string   
2         Frequency            string   
3              Year            string   

                                ParameterDescription  ParameterIsRequiredFlag  \
0                                 Type of investment                        0   
1                   Component of changes in position                        0   
2  A - Annual, QNSA - Quarterly not seasonally ad...                        0   
3                                     Year requested                        0   

  ParameterDefaultValue  MultipleAcceptedFlag AllValue  
0                   ALL                     1      ALL  
1                   ALL                     1      ALL  
2                   ALL                     1      ALL  
3                   ALL                     1      ALL

Get Type of Investment

In [62]:
# year_IIP = beaapi.get_parameter_values(beakey, 'IIP', 'Year') #if we wanted to get list of years
iip_Type = beaapi.get_parameter_values(beakey, 'IIP', 'TypeOfInvestment')
type_result = iip_Type[iip_Type["Desc"].str.contains(type_desc.lower())] #Note descriptions don't exaclty match lines so make lower case
display(type_result)
type_of_inv = type_result.iloc[0,0]

Key                                               Desc
343  NetExclFinDeriv  U.S. net international investment position exc...

Get data

In [63]:
tbl = beaapi.get_data(beakey, "IIP", AreaOrCountry="AllCountries",TypeOfInvestment=type_of_inv,Year="2020",Frequency="QNSA")
tbl.head()

TypeOfInvestment Component Frequency  Year                    TimeSeriesId  \
0  NetExclFinDeriv       Pos      QNSA  2020  TSI_IipNetExclFinDerivPos_QNSA   
1  NetExclFinDeriv       Pos      QNSA  2020  TSI_IipNetExclFinDerivPos_QNSA   
2  NetExclFinDeriv       Pos      QNSA  2020  TSI_IipNetExclFinDerivPos_QNSA   
3  NetExclFinDeriv       Pos      QNSA  2020  TSI_IipNetExclFinDerivPos_QNSA   

                               TimeSeriesDescription TimePeriod CL_UNIT  \
0  U.S. net international investment position exc...     2020Q1     USD   
1  U.S. net international investment position exc...     2020Q2     USD   
2  U.S. net international investment position exc...     2020Q3     USD   
3  U.S. net international investment position exc...     2020Q4     USD   

  UNIT_MULT  DataValue NoteRef  
0         6  -12150967          
1         6  -13018024          
2         6  -13794338          
3         6  -14004615

### Direct Investment & MNEs
See the separate guide iTables_MNE

### Regional

In [64]:
# Regional actually has the table name at the end of the description, so no need for get_table_param_from_desc()!
table_param_val= "SQGDP1"
year="2021"

print("What parameters are needed")
display(param_infos["Regional"])

What parameters are needed


ParameterName ParameterDataType  \
0       GeoFips            string   
1      LineCode           integer   
2     TableName            string   
3          Year            string   

                                ParameterDescription  ParameterIsRequiredFlag  \
0  Comma-delimited list of 5-character geographic...                        1   
1              Line code for a statistic or industry                        1   
2       Regional income or product table to retrieve                        1   
3  Comma-delimted list of years; LAST5 for latest...                        0   

   MultipleAcceptedFlag ParameterDefaultValue  
0                     1                  <NA>  
1                     0                  <NA>  
2                     0                        
3                     1                 LAST5

In [ ]:
linecode="ALL" # Mentioned in PDF
# Otherwise, could do
# pos_line_codes = beaapi.get_parameter_values_filtered(beakey, "Regional", "LineCode", TableName=table_param_val)
# display(pos_line_codes)

geo_fips = "00000" # Note: Not mentioned in docs, but common. See pdf for other common values
# Otherwise, could do
# pos_geo_codes = beaapi.get_parameter_values_filtered(beakey, "Regional", "GeoFips", TableName=table_param_val, LineCode=linecode)
# display(pos_geo_codes)

Get data

In [66]:
tbl = beaapi.get_data(beakey, "Regional", TableName=table_param_val, Year=year, GeoFips=geo_fips, LineCode=linecode)
tbl.head()

Code GeoFips        GeoName TimePeriod  \
0  SQGDP1-3   00000  United States     2021Q1   
1  SQGDP1-3   00000  United States     2021Q2   
2  SQGDP1-3   00000  United States     2021Q3   
3  SQGDP1-3   00000  United States     2021Q4   
4  SQGDP1-1   00000  United States     2021Q1   

                                        Description  \
0  Current-dollar GDP (millions of current dollars)   
1  Current-dollar GDP (millions of current dollars)   
2  Current-dollar GDP (millions of current dollars)   
3  Current-dollar GDP (millions of current dollars)   
4     Real GDP (millions of chained 2012 dollars)     

                            CL_UNIT  UNIT_MULT   DataValue  
0       Millions of current dollars          6  22038226.0  
1       Millions of current dollars          6  22740959.0  
2       Millions of current dollars          6  23202344.0  
3       Millions of current dollars          6  24002815.0  
4  Millions of chained 2012 dollars          6  19055655.0